# SLB Test Tool

## Setup

### Install Selenium

In [1]:
from IPython.display import clear_output

%pip install selenium 
clear_output(wait=True)

%pip install webdriver_manager 
clear_output(wait=True)

%pip install beautifulsoup4
clear_output(wait=True)

%pip install pandas 
clear_output(wait=True)

%pip install ipywidgets
clear_output()

print("All packages installed")

All packages installed


In [2]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

from bs4 import BeautifulSoup
from collections import defaultdict
from xml.sax.saxutils import quoteattr
from IPython.display import display, Javascript
from ipywidgets import HTML

import ipywidgets as widgets
import pandas as pd
import time
import json

prefs = {"profile.default_content_setting_values.notifications": 1}
chrome_options = Options()
#chrome_options.add_argument("--headless")
chrome_options.add_experimental_option("prefs", prefs)

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)
driver.quit()

def new_window(url, wait=0):
    driver = webdriver.Chrome(service=service, options=chrome_options)
    driver.get(url)
    time.sleep(wait)
    return driver

def highlight_element(driver, selector):
    script = f"""
    var element = document.evaluate('{selector}', document, null, XPathResult.FIRST_ORDERED_NODE_TYPE, null).singleNodeValue;
    if (element) {{
        var originalStyle = element.style.border;
        var count = 0;
        var interval = setInterval(function() {{
            count += 1;
            element.style.border = count % 2 ? '3px solid red' : '';
            if (count > 9) {{
                clearInterval(interval);
                element.style.border = originalStyle;
            }}
        }}, 500);
    }}
    """
    try:
        driver.execute_script(script)
    except Exception as e:
        print("Error while highlighting the element:", e)


clear_output()
print("Done!")

Done!


## Source Code

### Generate Objects Keys

In [3]:
def generate_key(tag, element, counter):
    base = {"input": "txt", "button": "btn", "a": "lnk", "img": "img"}
    prefix = base.get(tag, "elem")

    parts = [prefix]
    
    txt = element.get_text(strip=True)
    if txt:
        parts.append(txt.replace(" ", "_"))
        return key_builder(parts, counter)
    
    if element.get('name'):
        parts.append(element['name'])
        return key_builder(parts, counter)
    
    if element.get('aria-label'):
        parts.append(element['aria-label'].replace(" ", "_"))
        return key_builder(parts, counter)

    if element.get('id'):
        parts.append(element['id'])
        return key_builder(parts, counter)
    
    if element.get('data-test-id'):
        parts.append(element['data-test-id'])
        return key_builder(parts, counter)
    
    return key_builder(parts, counter)


def key_builder(parts, counter):
    key = '_'.join(filter(None, parts)).lower().replace("-", "_")
    counter[key] += 1
    return key if counter[key] == 1 else f"{key}_{counter[key]}"

print("Done!")

Done!


### Generate Objects Selectors

In [4]:
def generate_selector(element, driver):
    selectors = []

    for attr in ['id', 'name', 'placeholder', 'aria-label', 'data-test-id', 'alt']:
        if element.get(attr):
            value = quoteattr(element[attr])
            selectors.append(f"@{attr}={value}")


    if selectors:
        xpath_selector = f"//{element.name}[{' and '.join(selectors)}]"
        elements_found = driver.find_elements("xpath", xpath_selector)
        if len(elements_found) == 1:
            return xpath_selector


    txt = element.get_text(strip=True)
    if txt:
        xpath_selector = f"//{element.name}[contains(text(), {quoteattr(txt)})]"
        elements_found = driver.find_elements("xpath", xpath_selector)
        if len(elements_found) == 1:
            return xpath_selector


    if element.get('class'):
        classes = '.'.join(element.get('class'))
        xpath_selector = f"//{element.name}[contains(@class, {quoteattr(classes)})]"
        elements_found = driver.find_elements("xpath", xpath_selector)
        if len(elements_found) == 1:
            return xpath_selector

    return None

print("Done!")

Done!


### Generate Objects Selectors

In [5]:

def read_page_objects_metadata(driver):

    html = driver.page_source.encode("utf-8")
    soup = BeautifulSoup(html, 'html.parser')
    elements = soup.find_all(['input', 'button', 'a', 'img'])

    counter = defaultdict(int)
    result = defaultdict(list)

    for element in elements:
        tag = element.name
        selector = generate_selector(element, driver)
        key = generate_key(tag, element, counter)

        result[tag].append({
            "key": key,
            "selector": selector,
            "attributes": {attr: element.get(attr) for attr in element.attrs},
            "text": element.get_text(strip=True),
            "visibility": "visible" if element.get('type') != 'hidden' else "invisible"
        })
    return result


## GUI Componets

### Toolbar

In [6]:

class Toolbar:
    def __init__(self):
        self.new_window_button = widgets.Button(description="New Window") 
        self.inspect_button = widgets.Button(description="Inspect") 
        self.filter_text = widgets.Text(placeholder='Type to filter...') 
        self.export_button = widgets.Button(description="Export Selected")
        self.content = widgets.HBox([self.new_window_button, 
                                   self.inspect_button, 
                                   self.filter_text, 
                                   self.export_button])

### Table

In [7]:

class Table:

    def __init__(self, df=None):
        self.content = widgets.HBox()

        self.export_event = None
        self.data_event = None
        self.web_event = None

        self.data = df if df is not None else pd.DataFrame(
            columns=["Alias", "Text", "Web", "Export", "Data", "Type", "Att"])
        self.reload()

    def try_fire_export_event(self, b, index):
        if self.export_event is not None:
            self.export_event(b, index)

    def try_fire_data_event(self, b, index):
        if self.data_event is not None:
            dt = self.data["Data"][index]
            self.data_event(dt, index)

    def try_fire_web_event(self, b, index):
        if self.web_event is not None:
            self.web_event(b, index)

    def reload(self, df=None):

        if df is not None: self.data = df
        df = self.data
        lines = len(self.data) + 1
        grid = widgets.GridspecLayout(lines, 3)

        for i in range(lines):
            if i == 0: 
                grid[i, 0] = widgets.HTML(value="<b>Alias</b>")
                grid[i, 1] = widgets.HTML(value="<b>Text</b>")
                continue
            
            cell_value_alias = getattr(df.iloc[i-1], "Alias", '') if i > 0 else ''
            cell_value_web = getattr(df.iloc[i-1], "Web", '') if i > 0 else ''
            cell_value_text = getattr(df.iloc[i-1], "Text", '') if i > 0 else ''

            pnl = widgets.HBox([
                widgets.Button(description="N", layout=widgets.Layout(width='50px', height='30px')),
                widgets.Button(tooltip=cell_value_web, icon='eye', layout=widgets.Layout(width='50px', height='30px')),
                widgets.Button(icon='table', layout=widgets.Layout(width='50px', height='30px'))
            ], layout=widgets.Layout(width='auto', height='auto', weight='0'))

            pnl.children[0].on_click(lambda b, index=i-1: self.try_fire_export_event(b, index))
            pnl.children[1].on_click(lambda b, index=i-1: self.try_fire_web_event(b, index))
            pnl.children[2].on_click(lambda b, index=i-1: self.try_fire_data_event(b, index))

            grid[i, 0] = widgets.Text(value=str(cell_value_alias), layout=widgets.Layout(width='auto', weight='2'))        
            grid[i, 1] = widgets.Text(value=str(cell_value_text), layout=widgets.Layout(width='auto', weight='3'), disabled=True)
            grid[i, 2] = pnl

        self.content.children = [grid]


### Properties

In [8]:
class Properties:
    
    def __init__(self, properties={}):        
        self.content = widgets.HBox()
        self.reload(properties)

    def reload(self, properties={}):
        num_props = len(properties)
        grid = widgets.GridspecLayout(num_props + 1, 2, width='490px')
        grid[0, 0] = widgets.HTML(value="<b>Property</b>", layout=widgets.Layout(width='80px'))
        grid[0, 1] = widgets.HTML(value="<b>Value</b>", layout=widgets.Layout(width='400px'))  

        i = 0
        for i, (key, value) in enumerate(properties.items(), start=1):
            grid[i, 0] = widgets.Text(value=key, layout=widgets.Layout(width='80px'), disabled=True, bold=True)
            grid[i, 1] = widgets.Text(value=str(value), layout=widgets.Layout(width='400px'))

        self.content.children = [grid]

# Sample
data = {
    'key': 'btn_back', 
    'selector': '//button[contains(text(), "Back")]', 
    'text': 'Back', 
    'visibility': 'visible', 
    'type': 'button', 
    'class': ['btn', 'action', 'focus-visible:ring-2', 'focus-visible:ring-offset-0', 'focus-visible:ring-blue-400', 'flex', 'justify-center', 'items-center', 'is-sm', 'is-primary']
}
props = Properties(data)
display(props.content)


### Window

In [9]:
class Window():

    def __init__(self): 

        self.dataframe = pd.DataFrame(
            columns=["Alias", "Text", "Web", "Export", "Data", "Type", "Att"])

        self.properties = Properties()
        self.toolbar = Toolbar()
        self.table = Table(self.dataframe)

        self.grid = widgets.GridspecLayout(1, 2)
        self.grid[0,0] = self.table.content
        #self.grid[0,1] = self.properties.content 


    def redraw(self):
        #Sclear_output(wait=True)
        display(self.toolbar.content)
        display(self.grid)

    def reload(self, df, props):
        self.dataframe = df
        self.properties.reload(props)
        self.table.reload(df)

## GUI Events

In [10]:
def on_new_window_clicked(b=None):
    global test_window
    try: test_window.quit()
    except: pass
    test_window = new_window("https://life.stg.wellzesta.com/login")


def on_inspect_clicked(b=None, window=None):
    
    global test_window
    if test_window is None:
        on_new_window_clicked()

    metadata = read_page_objects_metadata(test_window)
    rows = []

    for tag_name, elements in metadata.items():
        for element in elements:

            att = element.pop('attributes', {})
            clazz =  att.pop('class', [])

            for k, v in att.items():
                element[k] = v

            element['class'] = clazz
            rows.append({
                "Export": False,
                "Alias": element.get('key', ''),
                "Web": element.get('selector', ''),
                "Type": tag_name,
                "Text": element.get('text', ''),
                "Data": element,
            })

    df = pd.DataFrame(rows)
    window.reload(df, {})
    print("Done!")


def start_playground():
    window = Window()
    toolbar = window.toolbar
    toolbar.new_window_button.on_click(on_new_window_clicked)
    toolbar.inspect_button.on_click(lambda b: on_inspect_clicked(b, window))

    props = window.properties
    def on_properties_clicked(b=None, index=-1):
        try:props.reload(b)
        except:pass

    window.table.data_event = on_properties_clicked
    display(props.content)
    window.redraw()


## Playground

In [11]:
README = """
When a DEV loves a QA
Can't keep his mind on any code
He'd debug all the errors
For the quality she's known

Yes, she's tough, oh so demanding
But he's the champ, codes so well
Even writes a testing framework
Just to make her world excel

Turns his back on his shortcuts
If they don't pass her test
When a DEV loves a QA
He's precise in every line
Crafting codes she envisioned

He'd give up all his shortcuts
And embrace the cleaner code
If she said that's the way
It ought to be

Though she's strict, he's so adept
Together they're a code-quality duet
"""

start_playground()

NameError: name 'df' is not defined